In [33]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import numpy as np

THIS CODE DEMONSTRATES HYPOTHESIS TESTING  TO FIND IF MEAN PRICE OF IPHONE BETWEEN AMAZON.CO.IN AND FLIPKART.COM

In [34]:
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

In [35]:
def get_data(URL,tag,attrs):
  ''' this function returns a list of price given url, tag and attrs'''
  webpage = requests.get(URL, headers=HEADERS)
  soup = BeautifulSoup(webpage.content, "html.parser")
  price = soup.find_all(tag, attrs=attrs)
  price_list =[]
  
  for p in price:
      price_list.append(p.text)
  return price_list

In [36]:
#get iphone price from amazon
amazon_url = "https://www.amazon.in/s?k=iphones&ref=nb_sb_noss_2"
amz_attrs={'class':'a-price-whole'}
amz_tag = "span"
amazon_price = get_data(amazon_url,amz_tag,amz_attrs)

In [37]:
amazon_price

['33,999',
 '78,133',
 '29,900',
 '47,900',
 '47,900',
 '33,999',
 '33,999',
 '38,999',
 '38,999',
 '97,900',
 '33,999',
 '38,999',
 '38,999',
 '95,900',
 '33,999',
 '33,999',
 '14,790',
 '69,900']

In [38]:
#get flipkart prices
flipkart_url = "https://www.flipkart.com/search?q=iphones&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"
flip_attrs={'class':'_30jeq3 _1_WHN1'}
flip_tag = "div"
flipkart_price = get_data(flipkart_url,flip_tag,flip_attrs)

In [39]:
flipkart_price

['₹59,999',
 '₹29,999',
 '₹64,999',
 '₹29,999',
 '₹54,999',
 '₹29,999',
 '₹64,999',
 '₹54,999',
 '₹54,999',
 '₹54,999',
 '₹59,999',
 '₹49,999',
 '₹54,999',
 '₹49,999',
 '₹49,999',
 '₹59,999',
 '₹49,999',
 '₹34,999',
 '₹34,999',
 '₹49,999',
 '₹34,999',
 '₹54,999',
 '₹59,999',
 '₹49,999']

In [40]:
len(flipkart_price)

24

In [41]:
len(amazon_price)

18

In [42]:
#create a dataframe with these columns
my_dict = {'amazon_rs':amazon_price,'flipkart_rs':flipkart_price}
data = pd.DataFrame.from_dict(my_dict,orient='index').T.fillna(0,axis=1)
data

,amazon_rs,flipkart_rs
0,"33,999","₹59,999"
1,"78,133","₹29,999"
2,"29,900","₹64,999"
3,"47,900","₹29,999"
4,"47,900","₹54,999"
5,"33,999","₹29,999"
6,"33,999","₹64,999"
7,"38,999","₹54,999"
8,"38,999","₹54,999"
9,"97,900","₹54,999"


In [43]:
data.head(2)

,amazon_rs,flipkart_rs
0,"33,999","₹59,999"
1,"78,133","₹29,999"


In [44]:
#data preprocessing
data['amazon_rs'] = data['amazon_rs'].str.replace(',','').astype(float)
data['flipkart_rs'] = data['flipkart_rs'].str.strip('₹')
data['flipkart_rs'] = data['flipkart_rs'].str.replace(',','').astype(float)

In [45]:
data.dtypes

amazon_rs      float64
flipkart_rs    float64
dtype: object

In [46]:
data.head(2)

,amazon_rs,flipkart_rs
0,33999.0,59999.0
1,78133.0,29999.0


In [47]:
#testing samples for normality
import scipy.stats as stats
print(stats.shapiro(data['amazon_rs'].iloc[:18]))
print(stats.shapiro(data['flipkart_rs']))

(0.7988661527633667, 0.0014639643486589193)
(0.8666502833366394, 0.004526052623987198)


The samples are not normal as p value < 0.05

In [48]:
import scipy.stats as stats
# perform two-sided test. You can use 'greater' or 'less' for one-sided test
stats.mannwhitneyu(x=data['amazon_rs'].iloc[:18], y=data['flipkart_rs'], alternative = 'two-sided')

MannwhitneyuResult(statistic=150.0, pvalue=0.09413903738421849)